<h1><u>SUDS/HYPO71 to MiniSEED/Seisan database converter</u></h1>
<p>This Python3 notebook demonstrates how to convert a set of files from the 1990s VDAP system. For each event, the following files were generated:</p>

<ul>
<li>YYMMDDCC.WVM - SUDS multiplexed waveform file (generated by XDETECT)
<li>YYMMDDCC.DMX - SUDS demultiplexed waveform file
<li>YYMMDDCC.PHA - HYPO71 phase pick file (generated by SUDSPick/BUDSPick)
<li>YYMMDDCC.PUN - HYPO71 summary file
</ul>

<p>SUDS files are converted to SAC files and then MiniSEED (direct conversion often doesn't work). MiniSEED files are then registered into a Seisan database, and a Seisan S-file (in the so-called "Nordic" format) is generated.</p>

<p>HYPO71 phase and summary files are converted and added into the Seisan S-file.</p>

<h3>Requirements:</h3>
<p>Win-SUDS Utilities (for programs demux.exe, irig.exe and sud2sac.exe) and ObsPy. Win-SUDS is included in this repository under WINAPPS/winsuds/. Win-SUDS must be run on a Windows PC (Windows 10 64-bit worked fine in December 2019).</p>

<h3>Multiple files?</h3>
<p>For multiple events (e.g. the entire MVO database of ~230,000 events), use the Python program vdap2seisanDB.py.</p>


Do all library imports first

In [1]:
import obspy.core as op
import glob
import matplotlib.pyplot as plt
import os

Set up all paths that will be used

In [28]:
cwd = os.getcwd()
WINSUDSPATH = os.path.join(cwd,'winsuds','bin')
SUDSbasename = '95080203'
WVMfile = SUDSbasename + '.WVM' # the original multiplexed PC-SUDS format event waveform file
DMXfile = SUDSbasename + '.DMX' # produced by demux.exe and irig.exe
SACbasename = SUDSbasename + '.sac' # produced  by sud2sac.exe
MSEEDfile1 = SUDSbasename + '.ms' # produced by sud2msed.exe
MSEEDfile2 = SUDSbasename + '.mseed' # produced by recombining SAC file
PHAfile = SUDSbasename + '.PHA' # this might exist if HYPO71 was run to locate the event
PUNfile = SUDSbasename + '.PUN' # this might exist if HYPO71 was run and generated a hypocenter
demux = os.path.join(WINSUDSPATH, 'demux.exe')
irig = os.path.join(WINSUDSPATH, 'irig.exe')
sud2sac = os.path.join(WINSUDSPATH, 'sud2sac.exe')
sud2msed = os.path.join(WINSUDSPATH, 'sud2msed.exe')
print('Paths setup okay')

Paths setup okay


Check that WVM files cannot be read by ObsPy

In [3]:
try:
    st = op.read(WVMfile)
except:
    print('ObsPy cannot read multiplexed SUDS format')

ObsPy cannot read multiplexed SUDS format


Demultiplex the WVM file

In [30]:
if os.path.exists(WVMfile): # if WVM file exists, demultiplex it
    print('Demultiplexing ' + WVMfile)
    os.system(demux + ' ' + WVMfile)
    print('Done')
else:
    print(WVMfile + ' does not exist')

Demultiplexing 95080203.WVM
Done


Check that DMX files cannot be read by ObsPy

In [5]:
try:
    st = op.read(DMXfile)
except:
    print('ObsPy cannot read demultiplexed SUDS format')

ObsPy cannot read demultiplexed SUDS format


Time correct the DMX file and convert to SAC files. 

Note that we found that sud2msed does not read trace headers, it just produces one trace from many and also has an incorrect start time. This is why we use sud2sac, which we have checked against sudsplot.exe and shows the correct trace headers, waveforms and times.

In [31]:
if os.path.exists(DMXfile): # if DMX file exists, time correct it then convert to SAC
    print('Time correcting ' + DMXfile)
    os.system(irig + ' ' + DMXfile)
    
    # This produces one file per trace
    print('Converting ' + DMXfile + ' to SAC files')
    os.system(sud2sac + ' ' + DMXfile)
    
    # This does not produce a valid Miniseed file - it misses channel headers and combines everything into 1 trace
    #print('Converting ' + DMXfile + ' to Miniseed file')
    #os.system(sud2msed + ' ' + DMXfile + ' ' + MSEEDfile1)
    
    print('Done')
else:
    print(DMXfile + ' does not exist')
    

Time correcting 95080203.DMX
Converting 95080203.DMX to SAC files
Done


Merge the SAC files into a single Miniseed file. 

In [32]:
# Now merge the SAC files into a single valid Miniseed file    
st = op.Stream()
sacfilelist = glob.glob(SACbasename + '-???')
if len(sacfilelist) > 0:
    for sacfile in sacfilelist:
        print('Combining ' + sacfile + ' into Miniseed file ' + MSEEDfile2)
        tr = op.read(sacfile)
        st = st + tr
    st.write(MSEEDfile2)
    print('Done')
else:
    print('No SAC files found matching ' + SACbasename + '*. Must have been bad DMX file ' + DMXfile)
    fbad = open('badDMXfileList.txt','a+')
    fbad.write(DMXfile + "\n")
    fbad.close()
    

Combining 95080203.sac-000 into Miniseed file 95080203.mseed
Combining 95080203.sac-001 into Miniseed file 95080203.mseed
Combining 95080203.sac-002 into Miniseed file 95080203.mseed
Combining 95080203.sac-003 into Miniseed file 95080203.mseed
Combining 95080203.sac-004 into Miniseed file 95080203.mseed
Combining 95080203.sac-005 into Miniseed file 95080203.mseed
Combining 95080203.sac-006 into Miniseed file 95080203.mseed
Combining 95080203.sac-007 into Miniseed file 95080203.mseed
Combining 95080203.sac-008 into Miniseed file 95080203.mseed
Combining 95080203.sac-009 into Miniseed file 95080203.mseed
Combining 95080203.sac-00A into Miniseed file 95080203.mseed
Combining 95080203.sac-00B into Miniseed file 95080203.mseed
Done


Optional sanity check to re-read the Miniseed file and plot it.

In [ ]:
st2 = op.read(MSEEDfile2)
st2.plot()

In [ ]:
# SUDSplot can be used to compare if Miniseed conversion was accurate, but will not show up from here
#os.system('sudsplot.exe ' + DMXfile)

Add this Miniseed event waveform file to the Seisan database.
The Miniseed file will be moved to [SEISANTOP]/WAV/[SEISANDB]/YYYY/MM
A corresponding Seisan S-file (event metadata file) will be created at [SEISANTOP]/REA/[SEISANDB]/YYYY/MM

The Seisan programs MAKEREA and AUTOREG are used here. Since they normally require user input, we create files to simulate this.

In [33]:
seisanDBname = 'ASNE2'
yy = SUDSbasename[0:2]
mm = SUDSbasename[2:4]
century = '19'
if yy < '80':
    century = '20'
yyyy = century + yy
fptr = open('makerea_wrapper.txt','w')
fptr.write(seisanDBname + '\n')
fptr.write(yyyy + mm + '\n')
fptr.write('\n')
fptr.write('BOTH\n')
fptr.close()
os.system('makerea < makerea_wrapper.txt')
os.system('dirf ' + MSEEDfile2)
fptr = open('autoreg_wrapper.txt','w')
fptr.write('L\n')
fptr.write('m\n')
fptr.write(seisanDBname + '\n')
fptr.write('gt\n')
fptr.close()
os.system('autoreg < autoreg_wrapper.txt')

0

In [34]:
def findLatestFile(dirpath):
    lists = os.listdir(dirpath)                                   
    lists.sort(key=lambda fn:os.path.getmtime(os.path.join(dirpath, fn)))
    return os.path.join(dirpath,lists[-1])

def displayFile(dirpath):
    if os.path.exists(dirpath):
        fptr = open(dirpath, 'r')
        str = fptr.read()
        print(str)
    else:
        print(dirpath + ' not found')

SFILE = findLatestFile(os.path.join(os.environ['SEISAN_TOP'],'REA',seisanDBname,yyyy,mm))
print(SFILE)
displayFile(SFILE)

C:\Seismo\REA\ASNE2\1995\08\02-0354-19L.S199508
 1995  8 2  354 19.4 L                                                         1
 ACTION:ARG 19-12-01 13:28 OP:gt   STATUS:               ID:19950802035419     I
 95080203.mseed                                                                6
 STAT SP IPHASW D HRMM SECON CODA AMPLIT PERI AZIMU VELO AIN AR TRES W  DIS CAZ7
                                                                                



If BUDSPICK generated a phase picks file (file extension *.PHA), add that into the S-file using the Seisan program HYPNOR

In [35]:
def appendPicks(hypnoroutfile, sfilepath):
    if os.path.exists(hypnoroutfile) and os.path.exists(sfilepath):
        pass
    else:
        print('Cannot combine files, one of the inputs does not exist')
    print("Appending " + hypnoroutfile + " to " + sfilepath)
    newlines = list()
    with open(sfilepath, "r") as fs:
        for line in fs:
            #print(line[-2])
            if line[-2] == '7':
                print("- Inserting " + hypnoroutfile)
                #fh = open(hypnoroutfile, 'r')
                with open(hypnoroutfile, 'r') as fh:
                    for hnline in fh:
                        if hnline[-2] != '1':
                            newlines.append(hnline)
                #str = fh.read()
                #fh.close()
                #newlines.append(str)
            else:
                newlines.append(line)
    fs2 = open(sfilepath, "w")
    for newline in newlines:
        fs2.write(newline)
    fs2.close()
    return
            
if os.path.exists(PHAfile):
    fptr = open('hypnor_wrapper.txt','w')
    fptr.write(PHAfile + '\n')
    fptr.write(century + '\n')
    fptr.close()
    displayFile('hypnor_wrapper.txt')
    os.system('hypnor < hypnor_wrapper.txt')
    if os.path.exists('hypnor.out'):
        print('Running HYPNOR on ' + PHAfile)
        appendPicks('hypnor.out', SFILE)
        print(' - Success')
    else:
        print(' - Failed')
else:
    print('No PHA file for ' + SUDSbasename)
displayFile(SFILE)

No PHA file for 95080203
 1995  8 2  354 19.4 L                                                         1
 ACTION:ARG 19-12-01 13:28 OP:gt   STATUS:               ID:19950802035419     I
 95080203.mseed                                                                6
 STAT SP IPHASW D HRMM SECON CODA AMPLIT PERI AZIMU VELO AIN AR TRES W  DIS CAZ7
                                                                                



In [38]:
def HSUMNOR(PUNfile):
    # Based on hsumnor.for in Seisan (which did not work, but this does!)
    # I should add something for the high accuracy lines too, as this only records origin to nearest tenth of second
    outstr = ' ' * 79 + '1'
    agency = 'MVO'
    if os.path.exists(PUNfile):
        #print('0123456789'*8)
        #displayFile(PUNfile)
        #print(' 1996  625 0337 32.9 L  61.588   3.495 15.1  TES 31 1.0 3.2LTES 3.0CTES 3.2LNAO1') # a line from a TEST file in Seisan
        f1 = open(PUNfile, 'r')
        with open(PUNFile, 'r') as f1:
            for line in f1:
                pass
            
        str = line
        #f1.read()
        #f1.close()
        yy = float(str[0:2].strip())
        mm = float(str[2:4].strip())
        dd = float(str[4:6].strip())
        hr = float(str[7:9].strip())
        mi = float(str[9:11].strip())
        sec = float(str[12:17].strip())
        lat = float(str[17:20].strip())
        mlat = float(str[21:26].strip())
        lon = float(str[27:30].strip())
        mlon = float(str[31:36].strip())
        depth = float(str[37:43].strip())
        magstr = str[45:49].strip()
        try:
            magstr = "%3.1f" % float(magstr)
        except:
            magstr = " " * 3
        nphase = float(str[50:53].strip())
        rms = float(str[62:66].strip())
        dlat = float(lat) + float(mlat)/60.0
        dlon = float(lon) + float(mlon)/60.0
        if yy < 80:
            cen = 20
        else:
            cen = 19
        newstr = ' %2d%02d %02d%02d %02d%02d %4.1f L ' % (cen,yy,mm,dd,hr,mi,sec)    
        newstr = newstr + '%7.3f%8.3f%5.1f  %s%3d%4.1f %s' % (dlat, dlon, depth, agency, nphase, rms, magstr)
        outstr = newstr + outstr[len(newstr):]
    return outstr

def insertHYPO71Summary(PUNfile, sfilepath):
    if os.path.exists(PUNfile) and os.path.exists(sfilepath):
        pass
    else:
        print('Cannot combine files, one of the inputs does not exist')
    print("Converting " + PUNfile + " and inserting into " + sfilepath)
    newlines = list()
    hypo71sumstr = HSUMNOR(PUNfile)
    if hypo71sumstr != "":
        newlines.append(hypo71sumstr + "\n")
    with open(sfilepath, "r") as fs:
        for line in fs:
            newlines.append(line)
    fs2 = open(sfilepath, "w")
    for newline in newlines:
        fs2.write(newline)
    fs2.close()
    return

if os.path.exists(PUNfile):
    
    try:
        insertHYPO71Summary(PUNfile, SFILE)
    except:
        print('Could not translate HYPO71 summary file %s' % PUNfile)
        displayFile(PUNfile)
    displayFile(SFILE)
    

Converting 95080203.PUN and inserting into C:\Seismo\REA\ASNE2\1995\08\02-0354-19L.S199508
Could not translate HYPO71 summary file 95080203.PUN
 DATE    ORIGIN    LAT N    LONG W    DEPTH    MAG NO GAP DMIN  RMS  ERH  ERZ QM
950802  354 46.73 16-38.95  62- 8.33  17.20         7 318  6.1 0.04  7.8 10.6 D1

 1995  8 2  354 19.4 L                                                         1
 ACTION:ARG 19-12-01 13:28 OP:gt   STATUS:               ID:19950802035419     I
 95080203.mseed                                                                6
 STAT SP IPHASW D HRMM SECON CODA AMPLIT PERI AZIMU VELO AIN AR TRES W  DIS CAZ7
                                                                                



In [41]:
def HSUMNOR(PUNfile):
    # Based on hsumnor.for in Seisan (which did not work, but this does!)
    # I should add something for the high accuracy lines too, as this only records origin to nearest tenth of second
    outstr = ' ' * 79 + '1'
    agency = 'MVO'
    if os.path.exists(PUNfile):
        #print('0123456789'*8)
        #displayFile(PUNfile)
        #print(' 1996  625 0337 32.9 L  61.588   3.495 15.1  TES 31 1.0 3.2LTES 3.0CTES 3.2LNAO1') # a line from a TEST file in Seisan
        with open(PUNfile, 'r') as f1:
            for line in f1:
                pass      
        str = line
        yy = float(str[0:2].strip())
        mm = float(str[2:4].strip())
        dd = float(str[4:6].strip())
        hr = float(str[7:9].strip())
        mi = float(str[9:11].strip())
        sec = float(str[12:17].strip())
        lat = float(str[17:20].strip())
        mlat = float(str[21:26].strip())
        lon = float(str[27:30].strip())
        mlon = float(str[31:36].strip())
        depth = float(str[37:43].strip())
        magstr = str[45:49].strip()
        try:
            magstr = "%3.1f" % float(magstr)
        except:
            magstr = " " * 3
        nphase = float(str[50:53].strip())
        rms = float(str[62:66].strip())
        dlat = float(lat) + float(mlat)/60.0
        dlon = float(lon) + float(mlon)/60.0
        if yy < 80:
            cen = 20
        else:
            cen = 19
        newstr = ' %2d%02d %02d%02d %02d%02d %4.1f L ' % (cen,yy,mm,dd,hr,mi,sec)    
        newstr = newstr + '%7.3f%8.3f%5.1f  %s%3d%4.1f %s' % (dlat, dlon, depth, agency, nphase, rms, magstr)
        outstr = newstr + outstr[len(newstr):]
    return outstr

def insertHYPO71Summary(PUNfile, sfilepath):
    if os.path.exists(PUNfile) and os.path.exists(sfilepath):
        pass
    else:
        print('Cannot combine files, one of the inputs does not exist')
    print("Converting " + PUNfile + " and inserting into " + sfilepath)
    newlines = list()
    hypo71sumstr = HSUMNOR(PUNfile)
    if hypo71sumstr != "":
        newlines.append(hypo71sumstr + "\n")
    with open(sfilepath, "r") as fs:
        for line in fs:
            newlines.append(line)
    fs2 = open(sfilepath, "w")
    for newline in newlines:
        fs2.write(newline)
    fs2.close()
    return

if os.path.exists(PUNfile):
    try:
        insertHYPO71Summary(PUNfile, SFILE)
    except:
        print('Could not translate HYPO71 summary file %s' % PUNfile)
        displayFile(PUNfile)
    displayFile(SFILE)
    

Converting 95080203.PUN and inserting into C:\Seismo\REA\ASNE2\1995\08\02-0354-19L.S199508
Converting 95080203.PUN and inserting into C:\Seismo\REA\ASNE2\1995\08\02-0354-19L.S199508
 1995 0802 0354 46.7 L  16.649  62.139 17.2  MVO  7 0.0                        1
 1995 0802 0354 46.7 L  16.649  62.139 17.2  MVO  7 0.0                        1
 1995  8 2  354 19.4 L                                                         1
 ACTION:ARG 19-12-01 13:28 OP:gt   STATUS:               ID:19950802035419     I
 95080203.mseed                                                                6
 STAT SP IPHASW D HRMM SECON CODA AMPLIT PERI AZIMU VELO AIN AR TRES W  DIS CAZ7
                                                                                



What remains to be done:
3. Some DMX files seem to be broken. Are these the ones copied over? Check 9508264I. Check ASNE_/1995/08/ * 1995/08 since these from two different copying processes.
6. make the list of number of files each day from the last event label of each day
7. Add DOI's for dataset and software & create github project
8. process bad???fileList.txt. For bad DMX files, I can check if WVM file exists and if so, convert. Otherwise, see if a valid miniseed file exists in gse_all on newton - but beware that it has not been shifted by 4 hours. Bad PUN files should now work, but I need to figure out how to add them. Would need to write in a way that works even if WVM and DMX file not found.
9. Go through whole summary hypo71 list, and try to associate with any event it overlaps with. Easiest to do this in ObsPy. 
10. Go through the sheets I am carrying around.

Modify script so that we do not run makerea each time

Things for later:
* do I need to apply 4 hour time shift (+4)?
* remove duplicate S-files
* merge with the student teaching cluster database. S-files should have same name (or be within a second). Grab the classification parts and add them. Or go the other way - adding any location/phase pick lines. Beware of 4 hour time difference.



In [54]:
print(DMXfile)

95080203.DMX


In [56]:
f = os.path.join('fred','bill','1995','07', DMXfile)
os.path.basename(f)[0:-4] + '.mseed'

'95080203.mseed'